good for sending rate, loss rate, and thorughput 

In [ ]:
import dpkt
import socket
import pandas as pd
import numpy as np

# Define IP addresses of the two clients and the server
client1_ip = '192.168.5.252'
client2_ip = '192.168.5.251'
server_ip = '192.168.5.249'

# Function to parse IP addresses
def inet_to_str(inet):
    try:
        return socket.inet_ntop(socket.AF_INET, inet)
    except ValueError:
        return None

# Initialize dictionaries to store data for each client
client1_data = {'Timestamp': [], 'Throughput': [], 'LossRate': [], 'RTT': [], 'Latency': [], 'SendingRate': [], 'CongestionWindow': []}
client2_data = {'Timestamp': [], 'Throughput': [], 'LossRate': [], 'RTT': [], 'Latency': [], 'SendingRate': [], 'CongestionWindow': []}

# Initialize variables to track sequence numbers for loss and RTT calculations
client1_seq_nums = set()
client2_seq_nums = set()
client1_rtt = {}
client2_rtt = {}
client1_total_packets = 0
client1_lost_packets = 0
client2_total_packets = 0
client2_lost_packets = 0

# Read the PCAP file
with open('/content/drive/MyDrive/modified2.pcap', 'rb') as f:
    pcap = dpkt.pcap.Reader(f)

    for ts, buf in pcap:
        eth = dpkt.ethernet.Ethernet(buf)

        # Ensure the packet has an IP layer
        if not isinstance(eth.data, dpkt.ip.IP):
            continue

        ip = eth.data

        # Ensure the packet has a TCP layer
        if not isinstance(ip.data, dpkt.tcp.TCP):
            continue

        tcp = ip.data

        # Extract source and destination IP addresses and ports
        src_ip = inet_to_str(ip.src)
        dst_ip = inet_to_str(ip.dst)
        src_port = tcp.sport
        dst_port = tcp.dport

        # Skip if IP conversion failed
        if not src_ip or not dst_ip:
            continue

        # Track packets sent from clients to the server
        if (src_ip == client1_ip and dst_ip == server_ip):
            seq_num = tcp.seq
            ack_num = tcp.ack
            timestamp = ts
            throughput = len(tcp.data)
            congestion_window = tcp.win

            # Calculate RTT
            if ack_num in client1_rtt:
                rtt = timestamp - client1_rtt.pop(ack_num)
                latency = rtt / 2  # Assuming symmetric paths for latency
            else:
                client1_rtt[seq_num] = timestamp
                rtt = None
                latency = None

            # Calculate loss
            if seq_num in client1_seq_nums:
                client1_lost_packets += 1
            client1_seq_nums.add(seq_num)
            client1_total_packets += 1

            # Calculate loss rate
            loss_rate = client1_lost_packets / client1_total_packets

            # Calculate sending rate (Throughput / duration)
            if client1_data['Timestamp']:
                time_diff = timestamp - client1_data['Timestamp'][-1]
                sending_rate = throughput / time_diff if time_diff > 0 else 0
            else:
                sending_rate = throughput

            # Append data to client 1 lists
            client1_data['Timestamp'].append(timestamp)
            client1_data['Throughput'].append(throughput)
            client1_data['LossRate'].append(loss_rate)
            client1_data['RTT'].append(rtt)
            client1_data['Latency'].append(latency)
            client1_data['SendingRate'].append(sending_rate)
            client1_data['CongestionWindow'].append(congestion_window)

        elif (src_ip == client2_ip and dst_ip == server_ip):
            seq_num = tcp.seq
            ack_num = tcp.ack
            timestamp = ts
            throughput = len(tcp.data)
            congestion_window = tcp.win

            # Calculate RTT
            if ack_num in client2_rtt:
                rtt = timestamp - client2_rtt.pop(ack_num)
                latency = rtt / 2  # Assuming symmetric paths for latency
            else:
                client2_rtt[seq_num] = timestamp
                rtt = None
                latency = None

            # Calculate loss
            if seq_num in client2_seq_nums:
                client2_lost_packets += 1
            client2_seq_nums.add(seq_num)
            client2_total_packets += 1

            # Calculate loss rate
            loss_rate = client2_lost_packets / client2_total_packets

            # Calculate sending rate (Throughput / duration)
            if client2_data['Timestamp']:
                time_diff = timestamp - client2_data['Timestamp'][-1]
                sending_rate = throughput / time_diff if time_diff > 0 else 0
            else:
                sending_rate = throughput

            # Append data to client 2 lists
            client2_data['Timestamp'].append(timestamp)
            client2_data['Throughput'].append(throughput)
            client2_data['LossRate'].append(loss_rate)
            client2_data['RTT'].append(rtt)
            client2_data['Latency'].append(latency)
            client2_data['SendingRate'].append(sending_rate)
            client2_data['CongestionWindow'].append(congestion_window)

# Convert lists to DataFrames
client1_df = pd.DataFrame(client1_data)
client2_df = pd.DataFrame(client2_data)

# Aggregate results per second for the first 110 seconds
def aggregate_data(df):
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
    df.set_index('Timestamp', inplace=True)
    df = df.resample('S').agg({
        'Throughput': 'sum',
        'LossRate': 'mean',
        'RTT': 'mean',
        'Latency': 'mean',
        'SendingRate': 'mean',
        'CongestionWindow': 'mean'
    }).fillna(0).reset_index()
    df = df[df['Timestamp'] < df['Timestamp'][0] + pd.Timedelta(seconds=110)]
    return df

client1_df = aggregate_data(client1_df)
client2_df = aggregate_data(client2_df)

# Debugging: Print first few rows of each DataFrame
print("Client 1 DataFrame head:")
print(client1_df.head())

print("Client 2 DataFrame head:")
print(client2_df.head())

# Write to CSV with formatted values
client1_df.to_csv('client1_data.csv', index=False, float_format='%.16f')
client2_df.to_csv('client2_data.csv', index=False, float_format='%.16f')


GOOd for latency extraction 

In [2]:
import dpkt
import socket
import pandas as pd

# Define IP addresses of the two clients and the server
client1_ip = '192.168.5.252'
client2_ip = '192.168.5.251'
server_ip = '192.168.5.249'

# Function to parse IP addresses
def inet_to_str(inet):
    try:
        return socket.inet_ntop(socket.AF_INET, inet)
    except ValueError:
        return None

# Initialize dictionaries to store data for each client
client1_data = {'Second': [], 'Latency': [], 'Throughput': [], 'Loss': [], 'CongestionWindow': [], 'RTT': [], 'SendingRate': []}
client2_data = {'Second': [], 'Latency': [], 'Throughput': [], 'Loss': [], 'CongestionWindow': [], 'RTT': [], 'SendingRate': []}

# Function to calculate RTT from a dictionary of timestamps
def calculate_rtt(ts_dict, ack_num, ts):
    if ack_num in ts_dict:
        return ts - ts_dict.pop(ack_num)
    return None

# Dictionaries to store timestamps for calculating RTT
client1_ts = {}
client2_ts = {}

# Time window and interval
time_window = 110

# Read the PCAPNG file
with open('/home/sit-research/Downloads/SWITCH-BBR-CUBIC-251-CUBIC-252.pcap', 'rb') as f:
    pcap = dpkt.pcap.Reader(f)

    start_time = None

    for ts, buf in pcap:
        if start_time is None:
            start_time = ts

        # Calculate the second relative to the start time
        second = int(ts - start_time)

        if second >= time_window:
            break

        eth = dpkt.ethernet.Ethernet(buf)
        
        # Make sure the packet has an IP layer
        if not isinstance(eth.data, dpkt.ip.IP):
            continue
        
        ip = eth.data
        
        # Make sure the packet has a TCP layer
        if not isinstance(ip.data, dpkt.tcp.TCP):
            continue
        
        tcp = ip.data
        
        # Extract source and destination IP addresses
        src_ip = inet_to_str(ip.src)
        dst_ip = inet_to_str(ip.dst)
        
        # Skip if IP conversion failed
        if not src_ip or not dst_ip:
            continue

        # Initialize metrics for the current second if not already done
        while len(client1_data['Second']) <= second:
            client1_data['Second'].append(len(client1_data['Second']))
            client1_data['Latency'].append([])
            client1_data['Throughput'].append(0)
            client1_data['Loss'].append(0)
            client1_data['CongestionWindow'].append([])
            client1_data['RTT'].append([])
            client1_data['SendingRate'].append([])

        while len(client2_data['Second']) <= second:
            client2_data['Second'].append(len(client2_data['Second']))
            client2_data['Latency'].append([])
            client2_data['Throughput'].append(0)
            client2_data['Loss'].append(0)
            client2_data['CongestionWindow'].append([])
            client2_data['RTT'].append([])
            client2_data['SendingRate'].append([])

        # Filter packets sent to the server from client 1
        if (src_ip == client1_ip and dst_ip == server_ip):
            throughput = len(tcp.data)
            loss = 1 if (tcp.flags & (dpkt.tcp.TH_RST | dpkt.tcp.TH_FIN)) else 0
            congestion_window = tcp.win
            rtt = None

            # Calculate RTT
            if tcp.flags & dpkt.tcp.TH_ACK:
                rtt = calculate_rtt(client1_ts, tcp.ack, ts)
                if rtt is not None:
                    client1_data['RTT'][second].append(rtt)
            elif tcp.flags & dpkt.tcp.TH_SYN:
                client1_ts[tcp.seq + 1] = ts  # SYN uses seq + 1 for ACK matching
            
            # Update metrics for client 1
            client1_data['Throughput'][second] += throughput
            client1_data['Loss'][second] += loss
            client1_data['CongestionWindow'][second].append(congestion_window)
            client1_data['Latency'][second].append(ts)
        
        # Filter packets sent to the server from client 2
        elif (src_ip == client2_ip and dst_ip == server_ip):
            throughput = len(tcp.data)
            loss = 1 if (tcp.flags & (dpkt.tcp.TH_RST | dpkt.tcp.TH_FIN)) else 0
            congestion_window = tcp.win
            rtt = None

            # Calculate RTT
            if tcp.flags & dpkt.tcp.TH_ACK:
                rtt = calculate_rtt(client2_ts, tcp.ack, ts)
                if rtt is not None:
                    client2_data['RTT'][second].append(rtt)
            elif tcp.flags & dpkt.tcp.TH_SYN:
                client2_ts[tcp.seq + 1] = ts  # SYN uses seq + 1 for ACK matching
            
            # Update metrics for client 2
            client2_data['Throughput'][second] += throughput
            client2_data['Loss'][second] += loss
            client2_data['CongestionWindow'][second].append(congestion_window)
            client2_data['Latency'][second].append(ts)

# Calculate averages and prepare the final data for CSV
for i in range(len(client1_data['Second'])):
    if client1_data['Latency'][i]:
        client1_data['Latency'][i] = (max(client1_data['Latency'][i]) - min(client1_data['Latency'][i])) / len(client1_data['Latency'][i])
    else:
        client1_data['Latency'][i] = None
    
    if client1_data['CongestionWindow'][i]:
        client1_data['CongestionWindow'][i] = sum(client1_data['CongestionWindow'][i]) / len(client1_data['CongestionWindow'][i])
    else:
        client1_data['CongestionWindow'][i] = None
    
    if client1_data['RTT'][i]:
        client1_data['RTT'][i] = sum(client1_data['RTT'][i]) / len(client1_data['RTT'][i])
    else:
        client1_data['RTT'][i] = None
    
    client1_data['SendingRate'][i] = client1_data['Throughput'][i] / 1 if client1_data['Throughput'][i] else None

for i in range(len(client2_data['Second'])):
    if client2_data['Latency'][i]:
        client2_data['Latency'][i] = (max(client2_data['Latency'][i]) - min(client2_data['Latency'][i])) / len(client2_data['Latency'][i])
    else:
        client2_data['Latency'][i] = None
    
    if client2_data['CongestionWindow'][i]:
        client2_data['CongestionWindow'][i] = sum(client2_data['CongestionWindow'][i]) / len(client2_data['CongestionWindow'][i])
    else:
        client2_data['CongestionWindow'][i] = None
    
    if client2_data['RTT'][i]:
        client2_data['RTT'][i] = sum(client2_data['RTT'][i]) / len(client2_data['RTT'][i])
    else:
        client2_data['RTT'][i] = None
    
    client2_data['SendingRate'][i] = client2_data['Throughput'][i] / 1 if client2_data['Throughput'][i] else None

# Convert lists to DataFrames
client1_df = pd.DataFrame(client1_data)
client2_df = pd.DataFrame(client2_data)

# Debugging: Print first few rows of each DataFrame
print("Client 1 DataFrame head:")
print(client1_df.head())
print("Client 2 DataFrame head:")
print(client2_df.head())

# Write to CSV
client1_df.to_csv('client1_data.csv', index=False)
client2_df.to_csv('client2_data.csv', index=False)



Client 1 DataFrame head:
   Second  Latency  Throughput  Loss  CongestionWindow   RTT  SendingRate
0       0      NaN           0     0               NaN  None          NaN
1       1      NaN           0     0               NaN  None          NaN
2       2      NaN           0     0               NaN  None          NaN
3       3      NaN           0     0               NaN  None          NaN
4       4      NaN           0     0               NaN  None          NaN
Client 2 DataFrame head:
   Second   Latency  Throughput  Loss  CongestionWindow   RTT  SendingRate
0       0       NaN           0     0               NaN  None          NaN
1       1       NaN           0     0               NaN  None          NaN
2       2       NaN           0     0               NaN  None          NaN
3       3       NaN           0     0               NaN  None          NaN
4       4  0.000192     2678979     0        570.535484  None    2678979.0
